# *1. Data Cleaning*

In [2]:
import pandas as pd
import numpy as np

In [3]:
df=pd.read_parquet("data.parquet")
print(df.head(1000))

                     banknifty     nifty  tte
time                                         
2021-01-01 09:15:00   0.286058  0.199729   27
2021-01-01 09:16:00   0.285381  0.200433   27
2021-01-01 09:17:00   0.284233  0.200004   27
2021-01-01 09:18:00   0.286104  0.199860   27
2021-01-01 09:19:00   0.285539  0.198951   27
...                        ...       ...  ...
2021-01-02 01:50:00   0.255651  0.178049   27
2021-01-02 01:51:00   0.255651  0.178049   27
2021-01-02 01:52:00   0.255651  0.178049   27
2021-01-02 01:53:00   0.255651  0.178049   27
2021-01-02 01:54:00   0.255651  0.178049   27

[1000 rows x 3 columns]


In [4]:
print("Dataset shape:", df.shape)

Dataset shape: (690512, 3)


In [5]:
print(df.isnull().sum())

banknifty    1500
nifty        2000
tte             0
dtype: int64


In [6]:
print(df.describe())

           banknifty          nifty            tte
count  689012.000000  688512.000000  690512.000000
mean        0.250156       0.179343      15.671884
std         0.057702       0.046139       8.664597
min         0.152732       0.089355       2.000000
25%         0.205307       0.140375       8.000000
50%         0.243977       0.181210      15.000000
75%         0.295293       0.215635      23.000000
max         0.460619       0.306952      31.000000


In [7]:
print(df.columns)

Index(['banknifty', 'nifty', 'tte'], dtype='object')


In [8]:
df = df.reset_index()

In [9]:
print(df.columns)

Index(['time', 'banknifty', 'nifty', 'tte'], dtype='object')


In [10]:
df = df.drop_duplicates('time')

In [11]:
print(df.describe())

           banknifty          nifty            tte
count  689012.000000  688512.000000  690512.000000
mean        0.250156       0.179343      15.671884
std         0.057702       0.046139       8.664597
min         0.152732       0.089355       2.000000
25%         0.205307       0.140375       8.000000
50%         0.243977       0.181210      15.000000
75%         0.295293       0.215635      23.000000
max         0.460619       0.306952      31.000000


In [12]:
print(df.head())

                 time  banknifty     nifty  tte
0 2021-01-01 09:15:00   0.286058  0.199729   27
1 2021-01-01 09:16:00   0.285381  0.200433   27
2 2021-01-01 09:17:00   0.284233  0.200004   27
3 2021-01-01 09:18:00   0.286104  0.199860   27
4 2021-01-01 09:19:00   0.285539  0.198951   27


In [13]:
df['time'] = pd.to_datetime(df['time'])

In [14]:
df['banknifty'].fillna(method='ffill', inplace=True)
df['nifty'].fillna(method='ffill', inplace=True)

In [15]:
print(df.isnull().sum())

time         0
banknifty    0
nifty        0
tte          0
dtype: int64


In [16]:
print(df.shape)

(690512, 4)


In [17]:
df = df[df['time'].dt.dayofweek < 5]
print(df.shape)

(494672, 4)


In [18]:
import datetime
start_time = datetime.time(9, 15)
end_time = datetime.time(15, 30)

In [19]:
df = df[df['time'].dt.time.between(start_time, end_time)]
print(df.shape)

(129720, 4)


# *2. Z-Score Trading System*

In [20]:
df.head()


,time,banknifty,nifty,tte
0,2021-01-01 09:15:00,0.286058,0.199729,27
1,2021-01-01 09:16:00,0.285381,0.200433,27
2,2021-01-01 09:17:00,0.284233,0.200004,27
3,2021-01-01 09:18:00,0.286104,0.199860,27
4,2021-01-01 09:19:00,0.285539,0.198951,27


### Define the variables

In [29]:
# ===============================
# EDITABLE PARAMETERS
# ===============================
entry_zs      = [1.5, 2.0, 2.5]
exit_zs       = [0, 0.5, 0.75]          # exit bands (always toward 0)
windows = [30, 120, 375]                # minutes
min_horizon    = 30                  # minutes
max_horizon    = 5 * 375             # ~5 trading days
cooldown_bars  = 0                 # optional: bars to wait after exit

In [30]:
# ===============================
# IMPORTS
# ===============================
import numpy as np
import pandas as pd

# ===============================
# RAW DATA SETUP (run once after you load your data)
# Assumes you already have a DataFrame named `df` with columns:
#   ['time', 'banknifty', 'nifty', 'tte']
# If your loaded df is named differently, adapt the next two lines accordingly.
# ===============================
df_raw = df.sort_values('time').drop_duplicates('time').copy()
df_raw['spread'] = df_raw['banknifty'] - df_raw['nifty']

# ===============================
# HELPER: (Re)build zscore for a given rolling window
# ===============================
def make_df_with_z(df0, window):
    """
    Build a fresh DataFrame with rolling z-score of the spread using `window`.
    Drops rows without full window or missing tte.
    """
    df = df0.copy()
    roll = df['spread'].rolling(window, min_periods=window)
    mu  = roll.mean()
    sd  = roll.std(ddof=0).replace(0, np.nan)
    df['zscore'] = (df['spread'] - mu) / sd
    df = df.dropna(subset=['zscore', 'tte']).reset_index(drop=True)
    return df

# ===============================
# TRADING ENGINE (mean reversion with CROSSING logic)
# ===============================
def run_trading_sim(df, entry_z, exit_z, min_horizon=30, max_horizon=1950, cooldown=0):
    """
    Long when z crosses DOWN through -entry_z; exit when it reverts UP to -exit_z.
    Short when z crosses UP through +entry_z; exit when it reverts DOWN to +exit_z.
    Entries/exits use CROSSING conditions to avoid flicker.
    """
    trades = []
    pos = None              # {'side': 'long'|'short', 'i0': entry_index}
    cooldown_left = 0

    for i in range(1, len(df)):
        z_prev, z = df['zscore'].iat[i-1], df['zscore'].iat[i]

        # Cooldown only blocks new entries (exits still allowed)
        if cooldown_left > 0:
            cooldown_left -= 1

        # ------- ENTRY -------
        if pos is None and cooldown_left == 0:
            # crossed UP through +entry_z -> SHORT
            if (z_prev <= entry_z) and (z > entry_z):
                pos = {'side': 'short', 'i0': i}
            # crossed DOWN through -entry_z -> LONG
            elif (z_prev >= -entry_z) and (z < -entry_z):
                pos = {'side': 'long', 'i0': i}

            # if entered, skip exit check this bar
            if pos is not None:
                continue

        # ------- EXIT -------
        if pos is not None:
            hold = i - pos['i0']

            # exits are toward zero, using CROSSING
            exit_long  = (pos['side'] == 'long'  and (z_prev < -exit_z) and (z >= -exit_z))
            exit_short = (pos['side'] == 'short' and (z_prev >  exit_z) and (z <=  exit_z))
            time_up    = hold >= max_horizon

            if hold >= min_horizon and (exit_long or exit_short or time_up):
                ent = df.iloc[pos['i0']]
                cur = df.iloc[i]

                # P/L per assignment:
                # PLValue(t) = spread(t) * (TTE(t) ** 0.7)
                pnl_ent = ent['spread'] * (ent['tte'] ** 0.7)
                pnl_cur = cur['spread'] * (cur['tte'] ** 0.7)
                pnl = (pnl_cur - pnl_ent) if pos['side'] == 'long' else (pnl_ent - pnl_cur)

                trades.append({
                    'side': pos['side'],
                    'entry_time': ent['time'],
                    'exit_time':  cur['time'],
                    'minutes': hold,
                    'entry_z': df['zscore'].iat[pos['i0']],
                    'exit_z':  z,
                    'pnl': pnl
                })
                pos = None
                cooldown_left = cooldown

    return pd.DataFrame(trades)

# ===============================
# METRICS (with winrate)
# ===============================
def evaluate(trades_df):
    if trades_df.empty:
        return {'trades': 0, 'pnl': 0.0, 'sharpe': np.nan, 'max_dd': np.nan, 'winrate': np.nan}

    # cumulative equity curve from per-trade P/Ls
    pnl_series = trades_df['pnl'].cumsum()

    # simple Sharpe proxy on per-trade P/Ls (for model selection)
    ret_series = trades_df['pnl']
    if ret_series.std(ddof=0) > 0:
        sharpe = ret_series.mean() / ret_series.std(ddof=0) * np.sqrt(252)
    else:
        sharpe = np.nan

    # max drawdown on cumulative P/Ls
    peak = pnl_series.cummax()
    drawdown = pnl_series - peak
    max_dd = drawdown.min()

    # winrate (exclude exact zero pnl from denom; change to >=0 if you prefer)
    wins = (trades_df['pnl'] > 0).sum()
    losses = (trades_df['pnl'] < 0).sum()
    denom = wins + losses
    winrate = (wins / denom) if denom > 0 else np.nan

    return {
        'trades': len(trades_df),
        'pnl': float(pnl_series.iloc[-1]),
        'sharpe': float(sharpe) if np.isfinite(sharpe) else np.nan,
        'max_dd': float(max_dd) if np.isfinite(max_dd) else np.nan,
        'winrate': float(winrate) if np.isfinite(winrate) else np.nan
    }

# ===============================
# EDITABLE PARAMETERS
# ===============================
entry_zs       = [1.5, 2.0, 2.5]
exit_zs        = [0.0, 0.5, 0.75]    # exit bands (always toward 0); 0.0 = zero-cross exit
rolling_window = 120                 # change and re-run the PREP line below to rebuild z
min_horizon    = 30                  # minutes
max_horizon    = 5 * 375             # ~5 trading days (use 375 min/day for India)
cooldown_bars  = 0                   # bars to wait after exit to avoid churn

# ===============================
# PREP (recompute z-score for the CURRENT rolling_window)
# ===============================
df = make_df_with_z(df_raw, rolling_window)

# ===============================
# RUN GRID & VIEW (single window)
# ===============================
windows = [30, 120, 375]
rows = []
for W in windows:
    dfW = make_df_with_z(df_raw, W)
    for ez in entry_zs:
        for xz in exit_zs:
            tr = run_trading_sim(dfW, entry_z=ez, exit_z=xz,
                                 min_horizon=min_horizon,
                                 max_horizon=max_horizon,
                                 cooldown=cooldown_bars)
            stats = evaluate(tr)
            rows.append({'window': W, 'entry_z': ez, 'exit_z': xz, **stats})

summary = pd.DataFrame(rows).sort_values(['window','pnl','sharpe'], ascending=[True, False, False])
summary

,window,entry_z,exit_z,trades,pnl,sharpe,max_dd,winrate
0,30,1.5,0.00,2343,78.319164,3.396260,-2.323060,0.776687
3,30,2.0,0.00,1965,76.323954,3.666034,-1.774451,0.789715
1,30,1.5,0.50,2245,68.180867,3.028223,-3.260890,0.738530
6,30,2.5,0.00,1398,62.420352,4.186740,-1.459489,0.816750
4,30,2.0,0.50,1907,62.245101,3.153679,-2.984681,0.766125
7,30,2.5,0.50,1374,54.621809,4.013283,-1.093095,0.791849
2,30,1.5,0.75,2175,54.147126,2.492685,-2.599553,0.704828
5,30,2.0,0.75,1856,51.485178,2.824055,-2.066911,0.744073
8,30,2.5,0.75,1347,44.999462,3.502940,-1.861883,0.767632
9,120,1.5,0.00,1503,88.231156,4.423657,-1.362327,0.882236
